Note: For easy understanding, read the code from bottom to top

In [1]:
from pathlib import Path
import numpy as np
import json
import os

In [2]:
#xywh to xcycwh coverter 
def xywh_to_xcycwh(bbox_array, width_of_image, height_of_image):

    x_min, y_min, width, height = bbox_array[0], bbox_array[1], bbox_array[2], bbox_array[3]            #here width and height refers to bbox

    x_center = x_min + (width/2)
    y_center = y_min + (height/2)

    #for normalizaton of dimensions
    xc_normalized = x_center/width_of_image
    yc_normalized = y_center/height_of_image
    width_normalized = width/width_of_image
    height_normalized = height/height_of_image
    
    return np.array([xc_normalized, yc_normalized, width_normalized, height_normalized], dtype=np.float32)
    

In [3]:
#extracting COCO data from json and convert it to YOLO format array
def coco_to_yolo(json_file):
    yolo_data = []
    with open(json_file, 'r') as opened_file:
        json_data = json.load(opened_file)

        image_width = json_data['image']['width']
        image_height = json_data['image']['height']

        for item in json_data['annotations']:
            local_array = []
            local_array.append(item['category_id'])
            local_array.extend(xywh_to_xcycwh(item['bbox'], image_width, image_height))
            yolo_data.append(local_array)

    return yolo_data

In [4]:
#json to .txt converter (this function takes json file, extract yolo format and save this info to .txt fiel)
def json_to_txt(json_file, label_folder):
    yolo_array = coco_to_yolo(json_file)

    file_name = os.path.splitext(os.path.basename(json_file))[0] + '.txt'
    file_path = label_folder / file_name

    with open(file_path, 'w') as txt_file:
        for array in yolo_array:
            txt_file.write(" ".join(map(str, array)) + '\n')

In [5]:
source_path = Path(r'C:\Users\hussa\OneDrive\Desktop\Project\Dataset\INFRA-3DRC_scene-23\INFRA-3DRC_scene-23\camera_01\camera_01__annotation')
destination_root = Path(r'C:\Users\hussa\OneDrive\Desktop\Project\Dataset\INFRA-3DRC_scene-23\INFRA-3DRC_scene-23\camera_01')

label_path  = Path(destination_root, 'labels')                  #creating a folder named labels in destination_root
label_path.mkdir(exist_ok = True)

for file in source_path.iterdir():
    json_to_txt(file, label_path)
    
